<a href="https://colab.research.google.com/github/QuinnMcGill/makeyourdatafair/blob/feature%2Fcallum-suppression/adult_suppression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Suppression Technique for Census Income Dataset

### Part 1: Install Dependencies

In [2]:
!pip install gower 'aif360[all]'

You can skip this next cell. It is mainly used for development and deletes all the notebook's variables.

In [3]:
!reset

Import packages and define functions to evaluate performance and indvidual fairness. Before you run the next cell, you'll need to put 'adult.data', 'adult.names' and 'adult.test' in '/usr/local/lib/python3.11/dist-packages/aif360/data/raw/adult'

Define performance evaluation helper function.

In [4]:
from IPython.display import Markdown, display
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score, classification_report, recall_score, f1_score

from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult

from group_fairness import eval_group_fairness

def eval_performance(y_test, y_pred):
    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Display metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/ma

Define individual fairness evaluation function.

In [5]:
from sklearn.neighbors import NearestNeighbors
import gower

def eval_ind_fairness(x_train, y_train, x_test, y_pred):
  # Compute Gower distance matrix for test samples w.r.t training data
  gower_distances = gower.gower_matrix(x_test, x_train)  # Shape: (num_test_samples, num_train_samples)

  # Find k nearest neighbors (excluding self)
  k = 5  # Adjust as needed
  neighbors = np.argsort(gower_distances, axis=1)[:, 1:k+1]  # Get indices of k nearest neighbors

  # Compute consistency score: Fraction of nearest neighbors with same prediction
  consistencies = []
  for i, neigh_indices in enumerate(neighbors):
      neighbor_preds = y_train[neigh_indices]  # Get predictions of k neighbors from training labels
      consistency = np.mean(neighbor_preds == y_pred[i])  # Fraction with same prediction
      consistencies.append(consistency)

  # Calculate overall consistency score
  individual_fairness_score = np.mean(consistencies)
  return individual_fairness_score

Define disparate impact evaluation function.

In [6]:
import pandas as pd

def eval_disparate_impact(x_test_priv_col, y_pred, unpriv_group_val=0, favourable_outcome=1):
  df = pd.DataFrame({
      'privileged_attribute': x_test_priv_col,
      'prediction': y_pred
  })

  # Calculate selection rates for protected and unprotected groups
  unpriveleged_group = df[df['privileged_attribute'] == unpriv_group_val]
  priveleged_group = df[df['privileged_attribute'] != unpriv_group_val]

  unpriveleged_selection_rate = np.mean(unpriveleged_group['prediction'] == favourable_outcome)
  priveleged_selection_rate = np.mean(priveleged_group['prediction'] == favourable_outcome)

  # Calculate disparate impact
  if priveleged_selection_rate == 0:
      disparate_impact = float('inf') if unpriveleged_selection_rate > 0 else 1.0
  else:
      disparate_impact = unpriveleged_selection_rate / priveleged_selection_rate

  return disparate_impact

Define counterfactual fairness evaluation function.

In [7]:
def evaluate_counterfactual_fairness_sex(model, X):
    """
    Evaluates counterfactual fairness by flipping the 'sex' attribute.
    """
    # Store the original predictions on the test set
    original_predictions = model.predict(StandardScaler().fit_transform(X))

    # Create a copy of the dataset
    X_counterfactual = X.copy()

    # Flip 'sex' (0 -> 1, 1 -> 0)
    if 'sex' in X.columns:
        X_counterfactual['sex'] = 1 - X_counterfactual['sex']
    else:
        raise KeyError(f"The sensitive attribute 'sex' is not present in the dataset.")

    # Get counterfactual predictions
    counterfactual_predictions = model.predict(StandardScaler().fit_transform(X_counterfactual))

    # Compare the original and counterfactual predictions
    comparison = pd.DataFrame({
        'original': original_predictions,
        'counterfactual': counterfactual_predictions,
        'same_decision': original_predictions == counterfactual_predictions
    })

    # Return the comparison dataframe
    return comparison

### Part 2: Load the Data

In [8]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])

np.random.seed(1)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/aif360/algorithms/preprocessing/optim_preproc_helpers/data_preproc_functions.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})


Split the data into training and test sets.

In [9]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.8], shuffle=True)

### Part 3: Train and Evaluate the Baseline

Not shown, refer to adult_reweighing.ipynb for baseline results.

### Part 4: Train and Evaluate the Model Trained on Suppressed Dataset

Drop the protected attribute column.

In [10]:
df_train_supp = dataset_orig_train.convert_to_dataframe()[0].drop('sex', axis=1).drop('Income Binary', axis=1)
df_test_supp = dataset_orig_test.convert_to_dataframe()[0].drop('sex', axis=1).drop('Income Binary', axis=1)

In [11]:
scale_orig = StandardScaler()
x_train_supp = scale_orig.fit_transform(df_train_supp)
y_train_supp = dataset_orig_train.labels.ravel()
w_train_supp = dataset_orig_train.instance_weights.ravel()

lmod_supp = LogisticRegression()
lmod_supp.fit(x_train_supp,
              y_train_supp,
              sample_weight=dataset_orig_train.instance_weights)

LogisticRegression()

In [12]:
x_test = scale_orig.fit_transform(df_test_supp)
y_test = dataset_orig_test.labels.ravel()

y_pred_supp = lmod_supp.predict(x_test)

In [13]:
# Evaluate performance of model trained on suppressed dataset
eval_performance(y_test, y_pred_supp)

Accuracy: 0.7912
Precision: 0.5838
Recall: 0.4282
F1 Score: 0.4940

Classification Report:
               precision    recall  f1-score   support

         0.0       0.84      0.90      0.87      7443
         1.0       0.58      0.43      0.49      2326

    accuracy                           0.79      9769
   macro avg       0.71      0.67      0.68      9769
weighted avg       0.78      0.79      0.78      9769



In [14]:
x_test_df = dataset_orig_test.convert_to_dataframe()[0]
disparate_impact = eval_disparate_impact(x_test_df['sex'], y_pred_supp)
print(f'Disparate Impact Score: {disparate_impact:.4f}')

Disparate Impact Score: 0.6496


In [15]:
individual_fairness_score = eval_ind_fairness(x_train_supp, y_train_supp, x_test, y_pred_supp)
print(f'Individual Fairness Consistency Score: {individual_fairness_score:.4f}')

Individual Fairness Consistency Score: 0.8028


In [18]:
# Model-level group fairness (reattach 'sex' to suppressed test set)
X_test_eval = df_test_supp.copy()  # suppressed features
X_test_eval['sex'] = x_test_df['sex']  # reattach 'sex'
supp_model_group_fairness = eval_group_fairness(
    X_test_eval, target='Income Binary', protected_attr='sex', mode='model', y_pred=y_pred_supp
)
print("\n⚖️ Model Group Fairness Metrics (Suppression):")
for metric, value in supp_model_group_fairness.items():
    print(f"{metric}: {value:.4f}")


⚖️ Model Group Fairness Metrics (Suppression):
Statistical Parity Difference: -0.0694
Disparate Impact: 0.6496
Demographic Parity: -0.0694


Counterfactual fairness is not applicable since we dropped the 'sex' attribute.